<a href="https://colab.research.google.com/github/Swatantra-D/Segmentation/blob/main/I_net_U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U_net code for image segmentation




In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle competitions download -c tgs-salt-identification-challenge
! unzip /content/tgs-salt-identification-challenge.zip
! unzip /content/train.zip -d trainee;

## The workflow goes like this


1.   Import libraries
2.   Import datasets
3.   Load in right format
4.   Preprocessing of images
5.   Prepare data for training
6.   *Define the model
7.   



#### Extra line to install libraries


#### 1.   Import libraries

Required Libraries:


1.   Data import and export libraries.
2.   DataTypes and analysis libaries.
3.   Image processing libraries.
4.   Machine learning libraries(normalize,Metrics)
5.   Deep learning libraries
6.   



In [1]:
# @markdown
import os
import random
import pandas as pd
import numpy as np
import wandb
import PIL
import torch
import torchvision
import tensorflow as tf
import matplotlib.pyplot as plt
import segmentation_models as sm

from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from IPython.display import clear_output
from tqdm import tqdm_notebook, tnrange
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

Segmentation Models: using `keras` framework.


#### Import datasets
Docstring: The datasets are in jpeg format and in multiple separate files.


Steps:
1.   Import the datasets
2.   Preprocess and normalize



In [11]:
# Path for images
datapath = r"C:\Users\Swata\Downloads\tgs-salt-identification-challenge\train"
# Checking number of images
ids = next(os.walk(r"C:\Users\Swata\Downloads\tgs-salt-identification-challenge\train\images"))[2] # list of names all images in the given path
print("No. of images = ", len(ids))

No. of images =  4000


In [6]:
# Create empty arrays
X = np.zeros((len(ids), 128, 128, 1), dtype=np.float32)
y = np.zeros((len(ids), 128, 128, 1), dtype=np.float32)

In [12]:
# Load tmages into arrays
for n,id_ in tqdm_notebook(enumerate(ids),total=len(ids)):

    #load images
    os.chdir(datapath + '\images') 
    x_img = img_to_array(load_img(id_,grayscale=True))
    x_img = resize(x_img, (128, 128, 1), mode = 'constant', preserve_range = True)
    #load masks
    os.chdir(datapath + '\masks') 
    mask = img_to_array(load_img(id_, grayscale=True))
    mask = resize(mask, (128, 128, 1), mode = 'constant', preserve_range = True)
    # Save images
    X[n] = x_img/255.0
    y[n] = mask/255.0


C:\Users\Swata\AppData\Local\Temp\ipykernel_1324\219810449.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n,id_ in tqdm_notebook(enumerate(ids),total=len(ids)):


  0%|          | 0/4000 [00:00<?, ?it/s]

C:\Users\Swata\anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [43]:
# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

#### Get the model

In [50]:
sm.set_framework('tf.keras')
base_model = sm.Unet('resnet34',classes=2, encoder_weights=None)
N = X.shape[-1]
inp = Input(shape=(None, None, N))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = base_model(l1)
model = Model(inp, out, name=base_model.name)

In [51]:
# Compile the model
model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])

In [48]:
X.shape[-1]

1

#### Fit the Model

In [53]:
model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    epochs=10,
    validation_data=(X_valid, y_valid),
)

InternalError: ignored

In [47]:
X.shape,y.shape,y_train.shape

((4000, 128, 128, 1), (4000, 128, 128, 1), (3600, 128, 128, 1))